# 開始爬蟲

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
# 搜尋目前上映那些電影，擷取出其ID資訊
name_list = []
author_list = []
intro_list = []
type_list = []
for i in range(1000):
    s = str(i).zfill(4)
    url = "https://reading.udn.com/v2/bookDesc.do?id=18" + s
    r = requests.get(url)
    web_content = r.text
    soup = BeautifulSoup(web_content, 'lxml')
    book_name = soup.find('h2')
    print(i)
    if(book_name != None):
        #print("書籍名稱:"+book_name.text.strip())
        name_list.append(book_name.text.strip())
        book_author1 = soup.find('div', class_ ="info_centerbox_line")
        book_author = book_author1.find('h4')
        author_list.append(book_author.text.strip())
        #print("作者:"+book_author.text.strip())
        book_intro1 = soup.find('div', class_ ="book_intro")
        book_intro = book_intro1.find('p')
        intro_list.append(book_intro.text.strip())
        #print("劇情簡介:"+book_intro.text.strip())
        book_type1 = soup.find('div', class_= "path")
        book_type = book_type1.find_next('a').find_next('a').find_next('a')
        type_list.append(book_type.text.strip())
        #print("分類:"+book_type.text.strip())
df_book_id = pd.DataFrame()
df_book_id["書籍名稱"]=name_list
df_book_id["作者"]=author_list
df_book_id["劇情簡介"]=intro_list
df_book_id["分類"]=type_list
#存檔案
df_book_id.to_csv("./final.csv",encoding="utf-8-sig")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


# 訓練

In [86]:
import pandas as pd
from sklearn.model_selection import train_test_split
import jieba
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
import jieba.analyse
from sklearn import metrics

In [113]:
data = pd.read_csv("final.csv",encoding = 'utf_8_sig')
data.head()
row,col = data.shape
tmp=[]
tmp=data['劇情簡介']
for i in range(row):
    tmp[i]=str(tmp[i])
data['劇情簡介']=tmp
data

D:\anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Unnamed: 0,書籍名稱,作者,劇情簡介,分類
0,0,到老師府辦桌：台北老家族的陳家菜,陳玠甫,台灣成長、美國留學、兩岸生活\r 擁有老靈魂的現代說書人\r 用一年的餐桌料理為您展示：...,生活休閒
1,1,從零開始：一段從失去中，重新找到家與愛的旅程,滕碧．洛克,沒有任何人事能治癒失去摯愛的傷，還有什麼能夠重新帶來生命的力量？一個觸動靈魂，論及悲傷探索、...,心靈勵志
2,2,太宰治請留步,黃文鉅,倘若你覺得自己三衰七敗，看不見明天的太陽，千萬、千萬不要急著去送死……痛必無獨，衰也有伴厭世...,文學
3,3,淡軼,文生,33個故事、33種人生，33也是耶穌死亡的年紀，但這數字同時也有種特別的味道。\r 整個世界...,文學
4,4,征服職場 Day by Day：小桌曆英文超給力系列,賴世雄教學團隊,nan,語言學習
5,5,心靈療癒 Day by Day：小桌曆英文超給力系列,賴世雄教學團隊,nan,語言學習
6,6,我每天吃十四顆藥，依舊相信會得到幸福,劉力穎,深入過痛苦，所以更懂得珍惜生活中的每一份感動和快樂。\r 經歷超過11年憂鬱症的百萬點閱...,心靈勵志
7,7,誰偷走了農地？：影響每一個人的台灣農業與農地公平正義,彭作奎,台灣糧食供應危機，多年來所發生的農地亂象，政治駕凌專業的歷史真相，本書一一揭露！民國八十九年...,社會人文
8,8,錢要投資 賺到退休 賺到自由健康：阮慕驊的人生體悟,阮慕驊,理財、理健康、理人生\r看人氣不老阿丹如何做\r 歷經2020年台灣股市由3月滑落谷底，又一...,商業財經
9,9,醒了就好：擁抱真愛的14段旅程,措尼仁波切,擁抱真愛，其實是個放鬆的過程，這個過程，是要連結最初的愛，超越習性的束縛，找回美好的自己。 ...,心靈勵志


In [114]:
def content_cut(text):
    specialChars = "！#$%^&*（）《  》【]「"
    text = text.replace("★" , "").replace("\r\n" , "").replace("！","").replace("《","").replace("》","").replace("，","").replace("/","").replace("（","").replace("）","")
    return " ".join(list(jieba.analyse.extract_tags(text,topK=2)))

data['分詞劇情'] = data['劇情簡介'].map(content_cut) 

In [115]:
data = { 'type':data["分類"] ,'name':data["書籍名稱"],'article':data['分詞劇情']}
df = pd.DataFrame(data)

df['key'] = df['name'].astype(str)+' '+df['type'].astype(str)+' '+df['article'].astype(str)

In [118]:
from sklearn.feature_extraction.text import TfidfVectorizer
def count(df): 
    tfidf2 = TfidfVectorizer()
    re = tfidf2.fit_transform(df['key'])
    return re

def knn(X,Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y['type'].str[0:2], test_size=0.2)
    clf=KNeighborsClassifier(n_neighbors=51)
    clf.fit(X_train,y_train)

    y_pred = clf.predict(X_test)
    y_test=y_test.values

    print("Test set score:",metrics.accuracy_score(y_test, y_pred))

tf = count(df)
knn(tf,df)

Test set score: 0.5882352941176471
